In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/playground-series-s3e26/train.csv')

In [3]:
train_df = pd.DataFrame(train_data)

In [4]:
train_df['Status'].value_counts()

Status
C     4965
D     2665
CL     275
Name: count, dtype: int64

In [5]:
train_df = train_df.drop('id', axis = 1)

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# sex, ascites, hepatomegaly, spiders, edema
train_df['Sex'] = le.fit_transform(train_df['Sex'])
train_df['Ascites'] = le.fit_transform(train_df['Ascites'])
train_df['Hepatomegaly'] = le.fit_transform(train_df['Hepatomegaly'])
train_df['Spiders'] = le.fit_transform(train_df['Spiders'])
train_df['Edema'] = le.fit_transform(train_df['Edema'])

In [7]:
train_df

,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,999,D-penicillamine,21532,1,0,0,0,0,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,2574,Placebo,19237,0,0,0,0,0,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,3428,Placebo,13727,0,0,1,1,2,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,2576,Placebo,18460,0,0,0,0,0,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,788,Placebo,16658,0,0,1,0,0,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,1166,D-penicillamine,16839,0,0,0,0,0,0.8,309.0,3.56,38.0,1629.0,79.05,224.0,344.0,9.9,2.0,C
7901,1492,Placebo,17031,0,0,1,0,0,0.9,260.0,3.43,62.0,1440.0,142.00,78.0,277.0,10.0,4.0,C
7902,1576,D-penicillamine,25873,0,0,0,1,1,2.0,225.0,3.19,51.0,933.0,69.75,62.0,200.0,12.7,2.0,D
7903,3584,D-penicillamine,22960,1,0,1,0,0,0.7,248.0,2.75,32.0,1003.0,57.35,118.0,221.0,10.6,4.0,D


In [8]:
from scipy import stats

# Assuming 'output' is the categorical output column and 'input' are numerical input columns
# 'data' is your DataFrame

output = train_df['Status']  # Replace 'output_column' with your output column name
input_numerical = train_df.iloc[: -1]  # Replace these with your numerical input columns

# Calculate Cramér's V for each input column against the output
cramer_vs = {}
for col in input_numerical.columns:
    confusion_matrix = pd.crosstab(input_numerical[col], output)
    chi2, _, _, _ = stats.chi2_contingency(confusion_matrix)
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    cramers_v = (phi2 / min(confusion_matrix.shape[0] - 1, confusion_matrix.shape[1] - 1))**0.5
    cramer_vs[col] = cramers_v

print(cramer_vs)


{'N_Days': 0.5095526688605646, 'Drug': 0.02735762446130133, 'Age': 0.4478999093019835, 'Sex': 0.13071481301924004, 'Ascites': 0.2766818744671928, 'Hepatomegaly': 0.39656780079016435, 'Spiders': 0.3243305909517575, 'Edema': 0.24154743288036049, 'Bilirubin': 0.44103040870173904, 'Cholesterol': 0.42258507758523123, 'Albumin': 0.3119974996764647, 'Copper': 0.4277447225758284, 'Alk_Phos': 0.4448115049481065, 'SGOT': 0.41530422850493387, 'Tryglicerides': 0.3322970358062611, 'Platelets': 0.4073388434112122, 'Prothrombin': 0.32652898998713636, 'Stage': 0.273381181916242, 'Status': 1.0}


In [9]:
train_df["New_Result"] = train_df["SGOT"]/train_df["Cholesterol"]
train_df["New_Result_2"] = train_df["SGOT"] * train_df["Cholesterol"]

In [10]:
X = pd.concat([train_df.iloc[:, :-3], train_df.iloc[:, -2], train_df.iloc[:, -1]], axis = 1)
y = train_df.iloc[:, -3]

In [11]:
X = X.drop(['Drug'], axis = 1)

In [12]:
X = X.drop(['Hepatomegaly'], axis =1)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [51]:
model = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=500, random_state=0, 
                      solver = 'sgd', activation = 'tanh'
                     ,learning_rate = 'invscaling', power_t = 0.47)

# Fitting the model
model.fit(X_train, y_train)       #0.45

MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50),
              learning_rate='invscaling', max_iter=500, power_t=0.47,
              random_state=0, solver='sgd')

In [52]:
y_pred = model.predict(X_test)

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7463630613535737

In [54]:
probabilities = model.predict_proba(X_test)
probabilities

array([[0.74930167, 0.09909661, 0.15160172],
       [0.41889102, 0.16125607, 0.41985291],
       [0.82376777, 0.1050073 , 0.07122493],
       ...,
       [0.3855033 , 0.23772629, 0.37677041],
       [0.71608163, 0.11683185, 0.16708652],
       [0.48909062, 0.30983571, 0.20107368]])

In [55]:
test_data = pd.read_csv('/kaggle/input/playground-series-s3e26/test.csv')
test_df = pd.DataFrame(test_data)

In [56]:
id = test_df['id']
test_df = test_df.drop('id', axis = 1)

test_df['Sex'] = le.fit_transform(test_df['Sex'])
test_df['Ascites'] = le.fit_transform(test_df['Ascites'])
test_df['Hepatomegaly'] = le.fit_transform(test_df['Hepatomegaly'])
test_df['Spiders'] = le.fit_transform(test_df['Spiders'])
test_df['Edema'] = le.fit_transform(test_df['Edema'])

In [57]:
test_df["New_Result"] = test_df["SGOT"]/test_df["Cholesterol"]
test_df["New_Result_2"] = test_df["SGOT"] * test_df["Cholesterol"]

test_df = test_df.drop(['Hepatomegaly'], axis = 1)

In [58]:
test_df = test_df.drop('Drug', axis = 1)
test_df = sc.transform(test_df)
test_predictions = model.predict_proba(test_df)
test_predictions = pd.DataFrame(test_predictions)

test_predictions

,0,1,2
0,0.552625,0.158929,0.288447
1,0.689861,0.153759,0.156380
2,0.346236,0.314099,0.339665
3,0.753859,0.119561,0.126580
4,0.729724,0.084272,0.186004
...,...,...,...
5266,0.708844,0.146427,0.144730
5267,0.584653,0.232815,0.182532
5268,0.486288,0.268531,0.245181
5269,0.855818,0.103803,0.040378


In [59]:
C = test_predictions.iloc[:, 0]
D = test_predictions.iloc[:, 1]
CL = test_predictions.iloc[:, 2]

In [60]:
submission_df = pd.DataFrame({'id' : id, 'Status_C' : C, "Status_CL": CL, 'Status_D' : D})
submission_df.to_csv('submission.csv', index = False)